In [1]:
from sift1m_dataset_builder import Builder
import tensorflow_datasets.public_api as tfds
from etils.epath import Path
import numpy as np
import time
import random

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import faiss
import torch

c:\Users\John\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = Path("B:/random ML datasets/dataset")
download_manager = tfds.download.DownloadManager(download_dir=path)

In [3]:
b = Builder()
split = b._split_generators(download_manager)

embeddingsGen = split["database"]

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:03<00:00,  3.14s/ url]
Extraction completed...: 0 file [00:03, ? file/s]
Dl Completed...: 100%|██████████| 1/1 [00:03<00:00,  3.15s/ url]


In [4]:
tensor_list = []
i = 0

for tup in embeddingsGen:
    dict = tup[1]
    tensor_list.append(torch.tensor(dict["embedding"]))
    i=i+1
    if i % 50000 == 0:
        print(i)

combined_tensor = torch.stack(tensor_list)

pq_input_array = combined_tensor.numpy()

50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000


In [5]:
print(pq_input_array)
print(pq_input_array.shape[1])



[[  0.  16.  35. ...  25.  23.   1.]
 [ 14.  35.  19. ...  11.  21.  33.]
 [  0.   1.   5. ...   4.  23.  10.]
 ...
 [ 30.  12.  12. ...  50.  10.   0.]
 [  0.   5.  12. ...   1.   2.  13.]
 [114.  31.   0. ...  25.  16.   0.]]
128


In [6]:
embeddingsGenTest = split["test"]

tensor_list_test = []
i = 0

for tup in embeddingsGenTest:
    dict = tup[1]
    tensor_list_test.append(torch.tensor(dict["embedding"]))
    i=i+1
    if i % 5000 == 0:
        print(i)

combined_tensor_test = torch.stack(tensor_list_test)

pq_input_array_test = combined_tensor_test.numpy()

5000
10000


In [7]:
dim = pq_input_array.shape[1]

pq = faiss.IndexFlatL2(dim)

In [8]:
pq.add(pq_input_array)

In [13]:
k = 10
distances, index = pq.search(pq_input_array_test[:10000], k)

In [9]:
idx_file = open("./temp.index", "w")
idx_file.truncate(0)
idx_file.close()
faiss.write_index(pq, "./temp.index")
file_size = os.path.getsize('./temp.index')
file_size

512000045

In [35]:
total_time = 0
iterations = 100

for x in range(1000):

    k = 100
    index = random.randint(0, 9999)

    input = np.empty((0,128))
    input = np.vstack([input,pq_input_array_test[index]])

    start_time = time.time()
    distances, index = pq.search(pq_input_array_test[:1], k)
    end_time = time.time()

    total_time += end_time - start_time


print("avg search time: ", total_time/iterations, " seconds")



avg search time:  0.18311583757400512  seconds


In [14]:
print(distances)
print(index)

[[58119.703]
 [49479.9  ]
 [29417.957]
 ...
 [48807.5  ]
 [48033.57 ]
 [37722.516]]
[[932085]
 [830117]
 [861530]
 ...
 [534149]
 [824828]
 [310710]]


In [10]:
# new recall-ish metric

k = 5 # number of returned neighbors for each embedding
t = 25 # top X number of closest neighbors


distances, index = pq.search(pq_input_array_test[:10000], k)


b = Builder()
split = b._split_generators(download_manager)

embeddingsGenTest = split["test"]

i = 0
relevantPercent = []

for tup in embeddingsGenTest:
    dict = tup[1]
    
    neighbors = dict["neighbors"]

    topIndexs = []
    for x in range(t):
        index_dict = neighbors[x]
        topIndexs.append(index_dict["index"])

    search_results = index[i]

    count = 0

    for x in search_results:
        if x in topIndexs:
            count+=1

    relevantPercent.append(count/k)
    
    i=i+1
        

print(sum(relevantPercent) / len(relevantPercent))

Dl Completed...: 0 url [00:00, ? url/s]
Extraction completed...: 0 file [00:00, ? file/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]
Dl Completed...: 0 url [00:00, ? url/s]


1.0


In [14]:
b = Builder()
split = b._split_generators(download_manager)

embeddingsGenTest = split["test"]

i = 0
recallAt5 = 0

for tup in embeddingsGenTest:
    dict = tup[1]
    
    neighbors = dict["neighbors"]

    topIndexs = []
    for x in range(1):
        index_dict = neighbors[x]
        topIndexs.append(index_dict["index"])

    search_results = index[i]
    if topIndexs[0] in search_results:
        recallAt5 += 1
    
    if i == 99:
        print("break")
        #break
    
    i=i+1
        

print(recallAt5)

Dl Completed...: 0 url [00:00, ? url/s]
Extraction completed...: 0 file [00:00, ? file/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]
Dl Completed...: 0 url [00:00, ? url/s]


break
10000


In [33]:
b = Builder()
split = b._split_generators(download_manager)

embeddingsGenTest = split["test"]

tup = next(embeddingsGenTest)
dict = tup[1]
print(dict)

Dl Completed...: 0 url [00:00, ? url/s]
Extraction completed...: 0 file [00:00, ? file/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]
Dl Completed...: 0 url [00:00, ? url/s]

{'index': 0, 'embedding': array([  1.,   3.,  11., 110.,  62.,  22.,   4.,   0.,  43.,  21.,  22.,
        18.,   6.,  28.,  64.,   9.,  11.,   1.,   0.,   0.,   1.,  40.,
       101.,  21.,  20.,   2.,   4.,   2.,   2.,   9.,  18.,  35.,   1.,
         1.,   7.,  25., 108., 116.,  63.,   2.,   0.,   0.,  11.,  74.,
        40., 101., 116.,   3.,  33.,   1.,   1.,  11.,  14.,  18., 116.,
       116.,  68.,  12.,   5.,   4.,   2.,   2.,   9., 102.,  17.,   3.,
        10.,  18.,   8.,  15.,  67.,  63.,  15.,   0.,  14., 116.,  80.,
         0.,   2.,  22.,  96.,  37.,  28.,  88.,  43.,   1.,   4.,  18.,
       116.,  51.,   5.,  11.,  32.,  14.,   8.,  23.,  44.,  17.,  12.,
         9.,   0.,   0.,  19.,  37.,  85.,  18.,  16., 104.,  22.,   6.,
         2.,  26.,  12.,  58.,  67.,  82.,  25.,  12.,   2.,   2.,  25.,
        18.,   8.,   2.,  19.,  42.,  48.,  11.], dtype=float32), 'neighbors': [{'index': 932085, 'distance': 232.87122}, {'index': 934876, 'distance': 234.71472}, {'index